Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [28]:
import pandas as pd

# Define DataFrames
bike_stations = pd.DataFrame({
    'Name': ['Station 1', 'Station 2', 'Station 3'],
    'Latitude': [40.748817, 40.761596, 40.764817],
    'Longitude': [-73.985428, -73.981618, -73.978508]
})

yelp_df = pd.DataFrame({
    'Station Name': ['Station 1', 'Station 2', 'Station 3'],
    'Rating': [4.5, 4.2, 4.7]
})


In [26]:
import pandas as pd

# Merge the DataFrames
combined_df = pd.merge(bike_stations, yelp_df, left_on='Name', right_on='Station Name', how='inner')

# Display the resulting combined DataFrame
print("Combined DataFrame:")
print(combined_df)

# Save the combined DataFrame to a CSV file
combined_df.to_csv('combined_bike_stations_yelp_data.csv', index=False)
print("Data successfully saved to 'combined_bike_stations_yelp_data.csv'.")

Combined DataFrame:
        Name   Latitude  Longitude Station Name  Rating
0  Station 1  40.748817 -73.985428    Station 1     4.5
1  Station 2  40.761596 -73.981618    Station 2     4.2
2  Station 3  40.764817 -73.978508    Station 3     4.7
Data successfully saved to 'combined_bike_stations_yelp_data.csv'.


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [29]:
import folium

# Debugging: Check column names
print("Bike Stations Columns:", bike_stations.columns)
print("Yelp Data Columns:", yelp_df.columns)

# Initialize the map centered around the average coordinates of bike stations
map_center = [bike_stations['Latitude'].mean(), bike_stations['Longitude'].mean()]
city_map = folium.Map(location=map_center, zoom_start=13)

# Add bike station markers
for _, station in bike_stations.iterrows():
    if 'Latitude' in station and 'Longitude' in station:
        folium.Marker(
            location=[station['Latitude'], station['Longitude']],
            popup=f"Station: {station['Name']}",
            icon=folium.Icon(color='blue')
        ).add_to(city_map)

# Add Yelp POI markers
for _, poi in yelp_df.iterrows():
    if 'Latitude' in poi and 'Longitude' in poi:
        folium.Marker(
            location=[poi['Latitude'], poi['Longitude']],
            popup=f"POI: {poi['POI Name']} ({poi['Rating']} stars, {poi['Review Count']} reviews)",
            icon=folium.Icon(color='green')
        ).add_to(city_map)

# Save and display the map
city_map.save('bike_stations_and_yelp_pois.html')
print("Map saved as 'bike_stations_and_yelp_pois.html'. Open it in a web browser to view!")


Bike Stations Columns: Index(['Name', 'Latitude', 'Longitude'], dtype='object')
Yelp Data Columns: Index(['Station Name', 'Rating'], dtype='object')
Map saved as 'bike_stations_and_yelp_pois.html'. Open it in a web browser to view!


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [30]:
import sqlite3
import pandas as pd
import os

# Ensure the 'data/' directory exists
data_dir = '/Users/erum/Statistical-Modelling-Project/data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Create SQLite connection and database file in the 'data/' directory
db_path = os.path.join(data_dir, "combined_data.db")  # Define the database file path
conn = sqlite3.connect(db_path)  # Connect to SQLite database
cursor = conn.cursor()

# Example DataFrame (bike station and POI data)
combined_df = pd.DataFrame({
    'Station Name': ['Station 1', 'Station 2', 'Station 3', 'Station 3'],
    'Latitude': [40.748817, 40.761596, 40.764817, 40.764817],
    'Longitude': [-73.985428, -73.981618, -73.978508, -73.978508],
    'POI Name': ['Joe\'s Pizza', 'Thai Corner', 'Barista Cafe', 'Sushi Spot'],
    'Rating': [4.5, 4.8, 4.6, 4.4],
    'Distance (m)': [250, 350, 200, 150],
    'Address': ['123 Main St', '789 Pine St', '101 Oak St', '202 Maple St']
})

# Step 1: Create a table in SQLite for the combined data
cursor.execute("""
CREATE TABLE IF NOT EXISTS combined_data (
    Station_Name TEXT,
    Latitude REAL,
    Longitude REAL,
    POI_Name TEXT,
    Rating REAL,
    Distance INTEGER,
    Address TEXT
)
""")

# Step 2: Insert data from the DataFrame into the SQLite table
for index, row in combined_df.iterrows():
    cursor.execute("""
    INSERT INTO combined_data (Station_Name, Latitude, Longitude, POI_Name, Rating, Distance, Address)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """, tuple(row))

# Commit changes
conn.commit()

# Step 3: Verify the data is inserted by querying the database
cursor.execute("SELECT * FROM combined_data")
results = cursor.fetchall()

# Print data for verification
print("Data in SQLite Database:")
for row in results:
    print(row)

# Close connection
conn.close()

# Optional: Print confirmation message
print(f"Database successfully created and stored at: {db_path}")

Data in SQLite Database:
('Station 1', 40.748817, -73.985428, "Joe's Pizza", 4.5, 250, '123 Main St')
('Station 2', 40.761596, -73.981618, 'Thai Corner', 4.8, 350, '789 Pine St')
('Station 3', 40.764817, -73.978508, 'Barista Cafe', 4.6, 200, '101 Oak St')
('Station 3', 40.764817, -73.978508, 'Sushi Spot', 4.4, 150, '202 Maple St')
('Station 1', 40.748817, -73.985428, "Joe's Pizza", 4.5, 250, '123 Main St')
('Station 2', 40.761596, -73.981618, 'Thai Corner', 4.8, 350, '789 Pine St')
('Station 3', 40.764817, -73.978508, 'Barista Cafe', 4.6, 200, '101 Oak St')
('Station 3', 40.764817, -73.978508, 'Sushi Spot', 4.4, 150, '202 Maple St')
('Station 1', 40.748817, -73.985428, "Joe's Pizza", 4.5, 250, '123 Main St')
('Station 2', 40.761596, -73.981618, 'Thai Corner', 4.8, 350, '789 Pine St')
('Station 3', 40.764817, -73.978508, 'Barista Cafe', 4.6, 200, '101 Oak St')
('Station 3', 40.764817, -73.978508, 'Sushi Spot', 4.4, 150, '202 Maple St')
Database successfully created and stored at: /Users

Look at the data before and after the join to validate your data.